Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels_nn = reformat(train_dataset, train_labels)
valid_dataset, valid_labels_nn = reformat(valid_dataset, valid_labels)
test_dataset, test_labels_nn = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000,)
Validation set (10000, 784) (10000,)
Test set (10000, 784) (10000,)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [8]:
# Logistic regularization

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

classifier=LogisticRegression(penalty='l2',solver='sag',multi_class="multinomial")

classifier.fit(train_dataset,train_labels)
predicted_labels=classifier.predict(test_dataset)
confusion_matrix(predicted_labels,test_labels)
accuracy_score(predicted_labels,test_labels)

0.90059999999999996

In [19]:
# The output of linear classifier on the validation set
print("The accuracy of logistic regression with L2\
      regularization and C=1.0 is %f"%accuracy_score(predicted_labels,test_labels))

The accuracy of logistic regression with L2      regularization and C=1.0 is 0.900600


In [5]:
# Neural network regularization
batch_size=128
hidden_layer_size=1024
num_steps = 3001
reg_param=0.001


sess1=tf.InteractiveSession()

# define variables
tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
tf_valid_dataset = tf.constant(valid_dataset)
tf_test_dataset = tf.constant(test_dataset)
tf_reg_param=tf.constant(reg_param)

#NETWORK CONSTRUCTION
#hidden layer
w_hidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
b_hidden = tf.Variable(tf.zeros([hidden_layer_size]))
l_hidden = tf.nn.relu(tf.matmul(tf_train_dataset, w_hidden) + b_hidden)

#read out layer
w_out = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
b_out = tf.Variable(tf.zeros([num_labels]))
l_out = tf.matmul(l_hidden, w_out) + b_out

cross_entropy=tf.nn.softmax_cross_entropy_with_logits(l_out, tf_train_labels)
loss = tf.reduce_mean(cross_entropy)+tf_reg_param*(tf.nn.l2_loss(w_hidden)
                                                   +tf.nn.l2_loss(w_out))

optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

train_prediction = tf.nn.softmax(l_out)
valid_prediction = tf.nn.softmax(tf.matmul(
        tf.nn.relu(tf.matmul(tf_valid_dataset, w_hidden) + b_hidden),w_out)+b_out)
test_prediction = tf.nn.softmax(tf.matmul(
        tf.nn.relu(tf.matmul(tf_test_dataset, w_hidden) + b_hidden),w_out)+b_out)

#EXECUTION
tf.initialize_all_variables().run()

print("Initialized")
for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]        
    batch_labels = train_labels_nn[offset:(offset + batch_size), :]
        
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = sess1.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
    if (step % 500 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels_nn))
print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels_nn))

sess1.close()


Initialized
Minibatch loss at step 0: 689.069824
Minibatch accuracy: 7.0%
Validation accuracy: 27.7%
Minibatch loss at step 500: 199.064819
Minibatch accuracy: 75.0%
Validation accuracy: 79.0%
Minibatch loss at step 1000: 115.994698
Minibatch accuracy: 80.5%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 68.747551
Minibatch accuracy: 86.7%
Validation accuracy: 82.8%
Minibatch loss at step 2000: 41.182217
Minibatch accuracy: 88.3%
Validation accuracy: 85.0%
Minibatch loss at step 2500: 25.136051
Minibatch accuracy: 87.5%
Validation accuracy: 85.5%
Minibatch loss at step 3000: 15.472307
Minibatch accuracy: 86.7%
Validation accuracy: 86.1%
Test accuracy: 93.0%


<h2>Answer 1</h2>
- Both Logistic Classifier and NN Classifier shows comparables performance on the available data set. Accuracys is slightly above 90% on the validation dataset.
- We can see about 2% accuracy improvement for NN Classifier after introduction of L2 Regularization for weights. 
- The tuning of L2 parameter for NN Classifier gave following results:
    - with L2 value = 0.01, score is ~90%
    - with L2 value = 0.1, score is 82.3%
    - with L2 value=0.05, score is 87.6%
    - with L2 value=0.005, score is 91.7%
    - with L2 value=0.001, score is 93.0%
    

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [29]:
# Neural network regularization
reduced_steps = 10
reg_param=0

reduced_train_labels=train_labels_nn[:200]
reduced_train_dataset=train_dataset[:200]


sess2=tf.InteractiveSession()

tf.initialize_all_variables().run()

print("Initialized")
for step in range(reduced_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (reduced_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = reduced_train_dataset[offset:(offset + batch_size), :]        
    batch_labels = reduced_train_labels[offset:(offset + batch_size), :]
        
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = sess2.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
    if (step % 2 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels_nn))
print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels_nn))

sess2.close()


Initialized
Minibatch loss at step 0: 628.422363
Minibatch accuracy: 11.7%
Validation accuracy: 33.0%
Minibatch loss at step 2: 1283.929077
Minibatch accuracy: 32.8%
Validation accuracy: 39.5%
Minibatch loss at step 4: 618.846558
Minibatch accuracy: 60.9%
Validation accuracy: 50.6%
Minibatch loss at step 6: 354.213318
Minibatch accuracy: 90.6%
Validation accuracy: 67.6%
Minibatch loss at step 8: 326.203949
Minibatch accuracy: 94.5%
Validation accuracy: 69.4%
Test accuracy: 74.0%


<h2> Answer 2 </h2>
- Validation and Test accuracy are low, while accuracy on minibatch is high.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [14]:
# Neural network regularization
batch_size=128
hidden_layer_size=1024
num_steps = 3001
reg_param=0.000000001
p_dropout=0.5
random_seed=999

sess3=tf.InteractiveSession()

# define variables
tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
tf_valid_dataset = tf.constant(valid_dataset)
tf_test_dataset = tf.constant(test_dataset)
tf_reg_param=tf.constant(reg_param)

#NETWORK CONSTRUCTION
#hidden layer
w_hidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
b_hidden = tf.Variable(tf.zeros([hidden_layer_size]))
l_hidden = tf.nn.relu(tf.matmul(tf_train_dataset, w_hidden) + b_hidden)

#dropout
l_dropout=tf.nn.dropout(x=l_hidden,keep_prob=p_dropout,seed=random_seed)
tf.Print(l_dropout,[l_dropout])

#read out layer
w_out = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
b_out = tf.Variable(tf.zeros([num_labels]))
l_out = tf.matmul(l_dropout, w_out) + b_out

cross_entropy=tf.nn.softmax_cross_entropy_with_logits(l_out, tf_train_labels)
loss = tf.reduce_mean(cross_entropy)+tf_reg_param*(tf.nn.l2_loss(w_hidden)
                                                   +tf.nn.l2_loss(w_out))

optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

train_prediction = tf.nn.softmax(l_out)
valid_prediction = tf.nn.softmax(tf.matmul(
        tf.nn.relu(tf.matmul(tf_valid_dataset, w_hidden) + b_hidden),w_out)+b_out)
test_prediction = tf.nn.softmax(tf.matmul(
        tf.nn.relu(tf.matmul(tf_test_dataset, w_hidden) + b_hidden),w_out)+b_out)

#EXECUTION
tf.initialize_all_variables().run()

print("Initialized")
for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]        
    batch_labels = train_labels_nn[offset:(offset + batch_size), :]
        
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = sess3.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
    if (step % 500 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels_nn))
print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels_nn))

sess3.close()


Exception AssertionError: AssertionError("Nesting violated for default stack of <type 'weakref'> objects",) in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x144026910>> ignored


Initialized
Minibatch loss at step 0: 548.395081
Minibatch accuracy: 10.9%
Validation accuracy: 27.4%
Minibatch loss at step 500: 38.227726
Minibatch accuracy: 75.0%
Validation accuracy: 79.8%
Minibatch loss at step 1000: 20.944315
Minibatch accuracy: 71.9%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 16.034319
Minibatch accuracy: 79.7%
Validation accuracy: 79.0%
Minibatch loss at step 2000: 5.480493
Minibatch accuracy: 80.5%
Validation accuracy: 78.5%
Minibatch loss at step 2500: 6.161688
Minibatch accuracy: 72.7%
Validation accuracy: 79.4%
Minibatch loss at step 3000: 3.012784
Minibatch accuracy: 71.9%
Validation accuracy: 79.5%
Test accuracy: 87.2%


In [16]:
# Neural network regularization
reduced_steps = 10
reg_param=0.001
p_dropout=0.1
random_seed=999

reduced_train_labels=train_labels_nn[:200]
reduced_train_dataset=train_dataset[:200]


sess4=tf.InteractiveSession()

tf.initialize_all_variables().run()

print("Initialized")
for step in range(reduced_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (reduced_train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = reduced_train_dataset[offset:(offset + batch_size), :]        
    batch_labels = reduced_train_labels[offset:(offset + batch_size), :]
        
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = sess4.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
    if (step % 2 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels_nn))
print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels_nn))

sess4.close()

Initialized
Minibatch loss at step 0: 477.737427
Minibatch accuracy: 13.3%
Validation accuracy: 31.5%
Minibatch loss at step 2: 1604.762085
Minibatch accuracy: 32.8%
Validation accuracy: 32.4%
Minibatch loss at step 4: 476.310486
Minibatch accuracy: 54.7%
Validation accuracy: 50.8%
Minibatch loss at step 6: 72.049065
Minibatch accuracy: 85.9%
Validation accuracy: 68.2%
Minibatch loss at step 8: 36.454388
Minibatch accuracy: 90.6%
Validation accuracy: 64.8%
Test accuracy: 74.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [33]:
# Neural network regularization
batch_size=128
hidden_layer_size=256
hidden_layer_size2=256

num_steps = 300
reg_param=0.0
p_dropout=1 # need to modify it. no dropout layer at the moment
random_seed=999

sess5=tf.InteractiveSession()

# define variables
tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
tf_valid_dataset = tf.constant(valid_dataset)
tf_test_dataset = tf.constant(test_dataset)
tf_reg_param=tf.constant(reg_param)

#NETWORK CONSTRUCTION
#hidden layer
w_hidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
b_hidden = tf.Variable(tf.zeros([hidden_layer_size]))
l_hidden = tf.nn.relu(tf.matmul(tf_train_dataset, w_hidden) + b_hidden)

#dropout TODO: experiment with dropout
#l_dropout=tf.nn.dropout(x=l_hidden,keep_prob=p_dropout,seed=random_seed)
#tf.Print(l_dropout,[l_dropout])

#hidden layer #2
w2_hidden = tf.Variable(tf.truncated_normal([hidden_layer_size, hidden_layer_size2]))
b2_hidden = tf.Variable(tf.zeros([hidden_layer_size2]))
l2_hidden = tf.nn.relu(tf.matmul(l_hidden, w2_hidden) + b2_hidden)

#read out layer
w_out = tf.Variable(tf.truncated_normal([hidden_layer_size2, num_labels]))
b_out = tf.Variable(tf.zeros([num_labels]))
l_out = tf.matmul(l2_hidden, w_out) + b_out

cross_entropy=tf.nn.softmax_cross_entropy_with_logits(l_out, tf_train_labels)

tf.Print(cross_entropy,[cross_entropy])
loss = tf.reduce_mean(cross_entropy)+tf_reg_param*(tf.nn.l2_loss(w_hidden)
                                                   + tf.nn.l2_loss(w2_hidden)
                                                   + tf.nn.l2_loss(w_out))

optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

train_prediction = tf.nn.softmax(l_out)
valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(
                tf.nn.relu(tf.matmul(tf_valid_dataset, w_hidden) + 
                           b_hidden),w2_hidden)+
                         b2_hidden),w_out)+ b_out)
test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(
                tf.nn.relu(tf.matmul(tf_test_dataset, w_hidden) + b_hidden),w2_hidden)+b2_hidden),w_out)+b_out)

#EXECUTION
tf.initialize_all_variables().run()

print("Initialized")
for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]        
    batch_labels = train_labels_nn[offset:(offset + batch_size), :]
        
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = sess5.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
    if (step % 1 == 0):
        print("Epoch id %d"%(offset/20000))
        print("Minibatch loss at step %d: %f" % (step, l))
        #print(cross_entropy)
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels_nn))
print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels_nn))

sess5.close()


Exception AssertionError: AssertionError("Nesting violated for default stack of <type 'weakref'> objects",) in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x16ca11bd0>> ignored


Initialized
Epoch id 0
Minibatch loss at step 0: 1583.802979
Minibatch accuracy: 4.7%
Validation accuracy: 10.0%
Epoch id 0
Minibatch loss at step 1: 402849.062500
Minibatch accuracy: 10.2%
Validation accuracy: 10.1%
Epoch id 0
Minibatch loss at step 2: 8988940.000000
Minibatch accuracy: 14.1%
Validation accuracy: 10.0%
Epoch id 0
Minibatch loss at step 3: 89661652992.000000
Minibatch accuracy: 5.5%
Validation accuracy: 10.2%
Epoch id 0
Minibatch loss at step 4: 10047690113024.000000
Minibatch accuracy: 10.2%
Validation accuracy: 10.1%
Epoch id 0
Minibatch loss at step 5: 226222510639065541378048.000000
Minibatch accuracy: 8.6%
Validation accuracy: 9.5%
Epoch id 0
Minibatch loss at step 6: nan
Minibatch accuracy: 8.6%
Validation accuracy: 10.0%
Epoch id 0
Minibatch loss at step 7: nan
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Epoch id 0
Minibatch loss at step 8: nan
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%
Epoch id 0
Minibatch loss at step 9: nan
Minibatch accura

KeyboardInterrupt: 

In [62]:
# Neural network regularization
n_hidden_1=256
n_hidden_2=256
reg_param=0.0001
p_dropout=1 # need to modify it. no dropout layer at the moment
random_seed=999
weight_std=1e-2


#general parameters
batch_size=128
image_size=28
num_steps = 10001
n_classes=10

# learning rate
learning_rate=0.5
decay_rate=0.9
decay_steps=1000
global_step = tf.Variable(0)  # count the number of steps taken.
decayed_learning_rate = tf.train.exponential_decay(learning_rate=learning_rate, global_step=global_step, 
                                                   decay_steps=decay_steps, decay_rate=decay_rate, staircase=True )

# define variables
tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, n_classes))
tf_valid_dataset = tf.constant(valid_dataset)
tf_test_dataset = tf.constant(test_dataset)
tf_reg_param=tf.constant(reg_param)

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.truncated_normal([image_size*image_size, n_hidden_1],stddev=weight_std)),
    'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2],stddev=weight_std)),
    'out': tf.Variable(tf.truncated_normal([n_hidden_2, n_classes],stddev=weight_std))
}
biases = {
    'b1': tf.Variable(tf.zeros([n_hidden_1])),
    'b2': tf.Variable(tf.zeros([n_hidden_2])),
    'out': tf.Variable(tf.zeros([n_classes]))
}

# Construct model
pred = multilayer_perceptron(tf_train_dataset, weights, biases)

# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, tf_train_labels))
optimizer=tf.train.GradientDescentOptimizer(learning_rate=decayed_learning_rate).minimize(loss,global_step=global_step)
train_prediction = tf.nn.softmax(pred)
valid_prediction=tf.nn.softmax(multilayer_perceptron(tf_valid_dataset,weights,biases))
test_prediction=tf.nn.softmax(multilayer_perceptron(tf_test_dataset,weights,biases))


In [63]:
# Initializing the variables
init = tf.initialize_all_variables()
print("Initialized")
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]        
        batch_labels = train_labels_nn[offset:(offset + batch_size), :]

        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l,predictions = sess.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 500 == 0):
            print("Epoch id %d"%(offset/20000))
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Learning rate %.1f" % (decayed_learning_rate.eval()))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels_nn))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels_nn))

    


Initialized
Epoch id 0
Minibatch loss at step 0: 2.302844
Learning rate 0.5
Minibatch accuracy: 5.5%
Validation accuracy: 10.0%
Epoch id 3
Minibatch loss at step 500: 0.368888
Learning rate 0.5
Minibatch accuracy: 88.3%
Validation accuracy: 84.5%
Epoch id 6
Minibatch loss at step 1000: 0.493060
Learning rate 0.4
Minibatch accuracy: 85.2%
Validation accuracy: 85.6%
Epoch id 9
Minibatch loss at step 1500: 0.277912
Learning rate 0.4
Minibatch accuracy: 92.2%
Validation accuracy: 87.6%
Epoch id 2
Minibatch loss at step 2000: 0.262710
Learning rate 0.4
Minibatch accuracy: 93.8%
Validation accuracy: 88.0%
Epoch id 6
Minibatch loss at step 2500: 0.299618
Learning rate 0.4
Minibatch accuracy: 89.8%
Validation accuracy: 88.5%
Epoch id 9
Minibatch loss at step 3000: 0.351559
Learning rate 0.4
Minibatch accuracy: 87.5%
Validation accuracy: 88.4%
Epoch id 2
Minibatch loss at step 3500: 0.395520
Learning rate 0.4
Minibatch accuracy: 88.3%
Validation accuracy: 89.0%
Epoch id 5
Minibatch loss at step

In [45]:
import matplotlib.pyplot as plt

rate=0.5
t = np.arange(0, 10000, 1000)
print(t)
s = rate*0.8**(t/1000)
plt.plot(t, s)

plt.xlabel('time (s)')
plt.ylabel('voltage (mV)')
plt.title('About as simple as it gets, folks')
plt.grid(True)
plt.savefig("test.png")
plt.show()

[   0 1000 2000 3000 4000 5000 6000 7000 8000 9000]
